# SQL

In [33]:
"""
Here we will be creating an SQL database to store disease signatures from iLINCS!

We will filter those signatures which belong to diseases - which are >9,000 signatures
from the iLINCS database.

Resources: 
    * http://www.ilincs.org/ilincs/APIinfo
"""

'\nHere we will be creating an SQL database to store disease signatures from iLINCS!\n\nWe will filter those signatures which belong to diseases - which are >9,000 signatures\nfrom the iLINCS database.\n\nResources: \n    * http://www.ilincs.org/ilincs/APIinfo\n'

## Create Table Dataset

In [23]:
# 1. Imports, Variables, Functions
import psycopg2
import pandas as pd
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# SQL variables
dbname = "ilincs"
user = "ddalton"
password = "Teclado$$$111"
host = "localhost"
path_data = "../../data/iLINCS/datasets.csv"
path_signature = "../../data/iLINCS/signatures.csv"
table_name = "datasets"
primary_key = "experiment"
int_columns = ["nsamples"]  # INT columns - rest TEXT
drop_table = True
columns_of_interest = ["geolink", "publink", "experiment", "organism", "description"]
ilincs_2_sql_columns = {
    "experiment": "dataset_id",
    "geolink": "geo_link",
    "publink": "pub_link",
    "organism": "organism",
    "description": "description",
}


# functions
def get_disease_datasets():
    """
    Get Disease Datasets
    Function to retrieve from those filtered signatures the datasetid

    Arguments:

    Return:
    datasetid: list()
        List of unique dataset id's
    """
    path_data = "../../data/iLINCS/signatures.csv"
    filter_df = lambda df: df["libraryid"] == "LIB_1"

    # Load Data
    try:
        df_data = pd.read_csv(path_data)
        logging.info("Data loaded successfully.")
    except FileNotFoundError:
        logging.error("Data file not found. Please check the file path.")
        exit()

    # filter disease signatures
    df_data = df_data[filter_df]

    return list(df_data["datasetid"].unique())


# 2. Load Data
try:
    df_data = pd.read_csv(path_data)
    logging.info("Data loaded successfully.")
except FileNotFoundError:
    logging.error("Data file not found. Please check the file path.")
    exit()

# get unique disease dataset id's
unique_datasetid = get_disease_datasets()

# filter disease signatures
# dataset id's refer to experiment
df_data = df_data[df_data["experiment"].isin(unique_datasetid)]

# filter columns of interest
df_data = df_data[columns_of_interest]

# assert we find all dataset id's
assert len(unique_datasetid) == len(
    df_data["experiment"].unique()
), "Error, Dataset Table does not conain all unique datasetid"

# get max length for each
max_lengths = [
    max([len(str(n)) for n in df_data[c].to_list()]) for c in df_data.columns
]

# Convert specified integer columns and handle NaN by replacing with 0
if any(c in df_data.columns for c in int_columns):
    for col in list(set(int_columns) & set(df_data.columns)):
        df_data[col] = (
            pd.to_numeric(df_data[col], errors="coerce").fillna(0).astype(int)
        )

# For other columns, replace NaN with None (which will become NULL in SQL)
for col in df_data.columns:
    if col not in int_columns:
        df_data[col] = df_data[col].where(pd.notnull(df_data[col]), None)

# Drop Duplicate for experiment column
df_data = df_data.drop_duplicates(subset="experiment", keep="first")

logging.info(f"Shape of filtered DataFrame: {df_data.shape}")

2023-12-13 15:51:58,318 - INFO - Data loaded successfully.
/tmp/ipykernel_809387/2114282074.py:45: DtypeWarning: Columns (0,3,7,8,9,10,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(path_data)
2023-12-13 15:51:58,782 - INFO - Data loaded successfully.
2023-12-13 15:51:58,833 - INFO - Shape of filtered DataFrame: (1087, 5)


In [20]:
"""for c in df_data.columns:
    print(f"######{c}######\n{df_data[c].value_counts()}")"""

'for c in df_data.columns:\n    print(f"######{c}######\n{df_data[c].value_counts()}")'

In [25]:
# 3. Connect with Database
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    logging.info("Connected to the database successfully.")
except psycopg2.OperationalError as e:
    logging.error(f"Unable to connect to the database: {e}")
    exit()

# 4. Create Cursor Object
cursor = conn.cursor()

# 5. Check if Table Exists and Delete Data if It Does
# Check if the table exists and drop it if it does
if drop_table:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE;")
        conn.commit()
        print(f"Table {table_name} dropped successfully if it existed.")
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()


cursor.execute(
    "SELECT EXISTS(SELECT * FROM information_schema.tables WHERE table_name=%s)",
    (table_name,),
)
table_exists = cursor.fetchone()[0]

if table_exists:
    try:
        cursor.execute(f"DELETE FROM {table_name};")
        conn.commit()
        logging.info(f"Existing data in table {table_name} deleted successfully.")
    except psycopg2.Error as e:
        logging.error(f"An error occurred while deleting data from the table: {e}")
        cursor.close()
        conn.close()
        exit()
else:
    # Create table if it does not exist
    column_text = ", ".join(
        f"{ilincs_2_sql_columns.get(c)} VARCHAR({n + 10})"
        if c not in int_columns
        else f"{ilincs_2_sql_columns.get(c)} INT"
        for c, n in zip(df_data.columns, max_lengths)
    )
    create_table_query = f"CREATE TABLE {table_name} ({column_text}, PRIMARY KEY({ilincs_2_sql_columns.get(primary_key)}));"
    try:
        cursor.execute(create_table_query)
        conn.commit()
        logging.info(f"Table {table_name} created successfully.")
    except psycopg2.Error as e:
        logging.error(f"An error occurred while creating the table: {e}")
        cursor.close()
        conn.close()
        exit()

2023-12-13 16:15:28,206 - INFO - Connected to the database successfully.
2023-12-13 16:15:28,215 - INFO - Table datasets created successfully.


Table datasets dropped successfully if it existed.


In [31]:
# 6. Dump Data into Table
data_tuples = list(df_data.itertuples(index=False, name=None))
insert_query = (
    f"INSERT INTO {table_name} ({', '.join([ilincs_2_sql_columns.get(c) for c in df_data.columns])}) VALUES (%s"
    + ", %s" * (len(df_data.columns) - 1)
    + ")"
)

try:
    with conn:
        with conn.cursor() as curs:
            for record in data_tuples:
                try:
                    curs.execute(insert_query, record)
                except psycopg2.Error as e:
                    logging.error(f"Error inserting record {record}: {e}")
                    # Optionally, you can break the loop after logging the first error
                    break
    logging.info(f"Data dumped into {table_name} successfully.")
except psycopg2.Error as e:
    logging.error(f"An error occurred while inserting data into the table: {e}")
    conn.rollback()
    exit()

2023-12-13 16:19:14,230 - INFO - Data dumped into datasets successfully.


## Create Signature Table

In [1]:
# 1. Imports, Variables, Functions
import psycopg2
import pandas as pd
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# SQL variables
dbname = "ilincs"
user = "ddalton"
password = "Teclado$$$111"
host = "localhost"
path_data = "../../data/iLINCS/signatures.csv"
table_name = "signatures"
primary_key = "signatureid"
int_columns = ["nCtrSamples", "nTrtSamples", "pubChemID"]  # INT columns - rest TEXT
drop_table = True
filter_df = lambda df: df["libraryid"] == "LIB_1"
reference_table = "datasets"
reference_key = "dataset_id"
foreign_key = "dataset_id"

columns_of_interest = [
    "signatureid",
    "datasetid",
    "level1",
    "level2",
    "tissue",
    "cellline",
]

ilincs_2_sql_columns = {
    "signatureid": "signature_id",
    "datasetid": "dataset_id",
    "level1": "condition_1",
    "level2": "condition_2",
    "tissue": "tissue",
    "cellline": "cell_line",
}

# 2. Load Data
try:
    df_data = pd.read_csv(path_data)
    logging.info("Data loaded successfully.")
except FileNotFoundError:
    logging.error("Data file not found. Please check the file path.")
    exit()

# filter disease signatures
# which are libraryid LIB_1
df_data = df_data[filter_df]

# filter columns of interest
df_data = df_data[columns_of_interest]

# get max length for each
max_lengths = [
    max([len(str(n)) for n in df_data[c].to_list()]) for c in df_data.columns
]

# Convert specified integer columns and handle NaN by replacing with 0
if any(c in df_data.columns for c in int_columns):
    for col in list(set(int_columns) & set(df_data.columns)):
        df_data[col] = (
            pd.to_numeric(df_data[col], errors="coerce").fillna(0).astype(int)
        )

# For other columns, replace NaN with None (which will become NULL in SQL)
for col in df_data.columns:
    if col not in int_columns:
        df_data[col] = df_data[col].where(pd.notnull(df_data[col]), None)

logging.info(f"Shape of filtered DataFrame: {df_data.shape}")

/tmp/ipykernel_810975/3235342070.py:37: DtypeWarning: Columns (0,3,7,8,9,10,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(path_data)
2023-12-13 16:38:04,516 - INFO - Data loaded successfully.
2023-12-13 16:38:04,557 - INFO - Shape of filtered DataFrame: (9097, 6)


In [3]:
# 3. Connect with Database
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    logging.info("Connected to the database successfully.")
except psycopg2.OperationalError as e:
    logging.error(f"Unable to connect to the database: {e}")
    exit()

# 4. Create Cursor Object
cursor = conn.cursor()

# 5. Check if Table Exists and Delete Data if It Does

# Check if the table exists and drop it if it does
if drop_table:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE;")
        conn.commit()
        print(f"Table {table_name} dropped successfully if it existed.")
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()


cursor.execute(
    "SELECT EXISTS(SELECT * FROM information_schema.tables WHERE table_name=%s)",
    (table_name,),
)
table_exists = cursor.fetchone()[0]

if table_exists:
    try:
        cursor.execute(f"DELETE FROM {table_name};")
        conn.commit()
        logging.info(f"Existing data in table {table_name} deleted successfully.")
    except psycopg2.Error as e:
        logging.error(f"An error occurred while deleting data from the table: {e}")
        cursor.close()
        conn.close()
        exit()
else:
    # Create table if it does not exist
    column_text = ", ".join(
        f"{ilincs_2_sql_columns.get(c)} VARCHAR({n + 10})"
        if c not in int_columns
        else f"{ilincs_2_sql_columns.get(c)} INT"
        for c, n in zip(df_data.columns, max_lengths)
    )
    create_table_query = f"CREATE TABLE {table_name} ({column_text}, PRIMARY KEY({ilincs_2_sql_columns.get(primary_key)}),FOREIGN KEY ({foreign_key}) REFERENCES {reference_table}({reference_key}));"
    try:
        cursor.execute(create_table_query)
        conn.commit()
        logging.info(f"Table {table_name} created successfully.")
    except psycopg2.Error as e:
        logging.error(f"An error occurred while creating the table: {e}")
        cursor.close()
        conn.close()
        exit()

2023-12-13 16:38:56,268 - INFO - Connected to the database successfully.
2023-12-13 16:38:56,275 - INFO - Table signatures created successfully.


Table signatures dropped successfully if it existed.


In [5]:
# 6. Dump Data into Table
data_tuples = list(df_data.itertuples(index=False, name=None))
insert_query = (
    f"INSERT INTO {table_name} ({', '.join([ilincs_2_sql_columns.get(c) for c in df_data.columns])}) VALUES (%s"
    + ", %s" * (len(df_data.columns) - 1)
    + ")"
)

try:
    with conn:
        with conn.cursor() as curs:
            for record in data_tuples:
                try:
                    curs.execute(insert_query, record)
                except psycopg2.Error as e:
                    logging.error(f"Error inserting record {record}: {e}")
                    # Optionally, you can break the loop after logging the first error
                    break
    logging.info(f"Data dumped into {table_name} successfully.")
except psycopg2.Error as e:
    logging.error(f"An error occurred while inserting data into the table: {e}")
    conn.rollback()
    exit()

2023-12-13 16:39:36,545 - INFO - Data dumped into signatures successfully.


## Create Table Signature Values

In [ ]:
# 1. Imports, Variables, Functions
import psycopg2
import pandas as pd
import logging, os
from tqdm import tqdm

# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# SQL variables
dbname = "ilincs"
user = "ddalton"
password = "Teclado$$$111"
host = "localhost"
path_data = "../../data/iLINCS/signature_vectors"
table_name = "signature_values"
primary_key = "signature_id_gene_id"
int_columns = []  # INT columns - rest TEXT
drop_table = True
foreign_key = "signature_id"
reference_table = "signatures"
reference_key = "signature_id"

columns_of_interest = [
    "signatureID",
    "ID_geneid",
    "Name_GeneSymbol",
    "Value_LogDiffExp",
    "Significance_pvalue",
]

ilincs_2_sql_columns = {
    "signatureID": "signature_id",
    "ID_geneid": "gene_id",
    "Name_GeneSymbol": "gene_name",
    "Value_LogDiffExp": "log_diff_exp",
    "Significance_pvalue": "p_value",
    "signature_id_gene_id": "signature_id_gene_id",
}


# functions
def get_disease_signatureids():
    """
    Get Disease Datasets
    Function to retrieve from those filtered signatures the datasetid

    Arguments:

    Return:
    datasetid: list()
        List of unique dataset id's
    """
    path_data = "../../data/iLINCS/signatures.csv"
    filter_df = lambda df: df["libraryid"] == "LIB_1"

    # Load Data
    try:
        df_data = pd.read_csv(path_data)
        logging.info("Data loaded successfully.")
    except FileNotFoundError:
        logging.error("Data file not found. Please check the file path.")
        exit()

    # filter disease signatures
    df_data = df_data[filter_df]

    return list(df_data["signatureid"].unique())

In [ ]:
# 2. Load Data

if os.path.exists("../../data/iLINCS/disease_signature_vectors.csv"):
    df_data = pd.read_csv("../../data/iLINCS/disease_signature_vectors.csv")
else:
    # get signature ids
    signature_ids = get_disease_signatureids()
    loop = 0
    # get data
    for signature_id in tqdm(signature_ids):
        if loop == 0:
            try:
                df_data = pd.read_csv(os.path.join(path_data, signature_id + ".csv"))
                loop = 1
            except FileNotFoundError:
                logging.error("Data file not found. Please check the file path.")
                exit()

        else:
            try:
                df = pd.read_csv(os.path.join(path_data, signature_id + ".csv"))
                df_data = pd.concat([df, df_data])
            except FileNotFoundError:
                logging.error("Data file not found. Please check the file path.")
                exit()

    # save dataframe
    try:
        df_data.to_csv("../../data/iLINCS/disease_signature_vectors.csv", index=False)
    except Exception as e:
        logging.error(f"Error saving data to csv: {e}")
        exit()

In [ ]:
df_data.shape

(144545011, 6)

In [14]:
# create primary key column
df_data["signature_id_gene_id"] = (
    df_data["signatureID"] + "_" + df_data["ID_geneid"].astype(str)
)

# filter columns of interest
df_data = df_data[columns_of_interest]

In [15]:
# get max length for each
max_lengths = [
    max([len(str(n)) for n in df_data[c].to_list()]) for c in df_data.columns
]

In [16]:
# 3. Connect with Database
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    logging.info("Connected to the database successfully.")
except psycopg2.OperationalError as e:
    logging.error(f"Unable to connect to the database: {e}")
    exit()

# 4. Create Cursor Object
cursor = conn.cursor()

# 5. Check if Table Exists and Delete Data if It Does

# Check if the table exists and drop it if it does
if drop_table:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE;")
        conn.commit()
        print(f"Table {table_name} dropped successfully if it existed.")
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")
        conn.rollback()


cursor.execute(
    "SELECT EXISTS(SELECT * FROM information_schema.tables WHERE table_name=%s)",
    (table_name,),
)
table_exists = cursor.fetchone()[0]

if table_exists:
    try:
        cursor.execute(f"DELETE FROM {table_name};")
        conn.commit()
        logging.info(f"Existing data in table {table_name} deleted successfully.")
    except psycopg2.Error as e:
        logging.error(f"An error occurred while deleting data from the table: {e}")
        cursor.close()
        conn.close()
        exit()
else:
    # Create table if it does not exist
    column_text = ", ".join(
        f"{ilincs_2_sql_columns.get(c)} VARCHAR({n + 10})"
        if c not in int_columns
        else f"{ilincs_2_sql_columns.get(c)} INT"
        for c, n in zip(df_data.columns, max_lengths)
    )
    create_table_query = f"CREATE TABLE {table_name} ({column_text}, PRIMARY KEY({ilincs_2_sql_columns.get(primary_key)}),FOREIGN KEY ({foreign_key}) REFERENCES {reference_table}({reference_key}));"
    try:
        cursor.execute(create_table_query)
        conn.commit()
        logging.info(f"Table {table_name} created successfully.")
    except psycopg2.Error as e:
        logging.error(f"An error occurred while creating the table: {e}")
        cursor.close()
        conn.close()
        exit()

2023-12-14 17:14:16,695 - INFO - Connected to the database successfully.
2023-12-14 17:14:16,699 - ERROR - An error occurred while creating the table: column "signature_id_gene_id" named in key does not exist
LINE 1: ...), log_diff_exp VARCHAR(33), p_value VARCHAR(33), PRIMARY KE...
                                                             ^



Table signature_values dropped successfully if it existed.


: 

In [21]:
signature_ids = get_disease_signatureids()

/tmp/ipykernel_810975/3237752548.py:17: DtypeWarning: Columns (0,3,7,8,9,10,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(path_data)
2023-12-13 18:25:11,755 - INFO - Data loaded successfully.


In [16]:
df = pd.read_csv("../../data/iLINCS/signature_vectors/GDS_1000.csv")

,signatureID,PROBE,ID_geneid,Name_GeneSymbol,Value_LogDiffExp,Significance_pvalue
0,GDS_1000,NaN,5320,PLA2G2A,-8.002820,5.012735e-14
1,GDS_1000,NaN,5321,PLA2G4A,7.830690,7.822211e-14
2,GDS_1000,NaN,92235,DUSP27,-9.207620,1.222107e-13
3,GDS_1000,NaN,9324,HMGN3,7.586360,2.091724e-13
4,GDS_1000,NaN,2697,GJA1,7.700330,2.495001e-13
...,...,...,...,...,...,...
19680,GDS_1000,NaN,388419,BTBD17,0.000170,9.989195e-01
19681,GDS_1000,NaN,10664,CTCF,-0.000133,9.990614e-01
19682,GDS_1000,NaN,150379,PNPLA5,0.000153,9.991050e-01
19683,GDS_1000,NaN,56901,NDUFA4L2,0.000107,9.992070e-01


In [ ]:
int_columns = ["ID_geneid"]
float_columns = ["Value_LogDiffExp", "Significance_pvalue"]

In [10]:
len(df_data["geneid"].to_list())

235982

## Query

In [7]:
# SQL variables
dbname = "ilincs"
user = "ddalton"
password = "Teclado$$$111"
host = "localhost"
table_name = "signatures"
columns_of_interest = {"ID_genid":"gene_id",
                       "",
                       ""}
# Connect to the database
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    print("Connected to the database successfully.")
except psycopg2.OperationalError as e:
    print(f"Unable to connect to the database: {e}")
    exit()

# Query the table
try:
    query = f"SELECT * FROM {table_name} LIMIT 10;"  # Adjust the query as needed
    query = f"SELECT * FROM {table_name} WHERE antibodytarget IS NOT NULL;"
    query = f"SELECT * FROM {table_name} WHERE experiment IS  NULL;"
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)
    print(df)
except Exception as e:
    print(f"An error occurred while querying the table: {e}")
finally:
    conn.close()

# The DataFrame 'df' now contains the first 10 rows of the table.

Connected to the database successfully.
     signature_id  dataset_id condition_1                     condition_2  \
0           GDS_1      GDS690      Caco-2                             T84   
1          GDS_10      GDS852     control                         stretch   
2         GDS_100  gdsGDS1454          6q  unknown_chromosomal_aberration   
3        GDS_1000  gdsGDS4296        A549                         COLO205   
4        GDS_1001  gdsGDS4296        A549                     DU_145(DTP)   
...           ...         ...         ...                             ...   
9092      GDS_994  gdsGDS4296        A498                        UACC_257   
9093      GDS_995  gdsGDS4296        A498                         UACC_62   
9094      GDS_996  gdsGDS4296        A498                           UO_31   
9095      GDS_997  gdsGDS4296        A549                            ACHN   
9096      GDS_999  gdsGDS4296        A549                        CCRF_CEM   

     tissue cell_line  
0      None

/tmp/ipykernel_810975/183834926.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
